In [1]:
import os
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from BassetModel import DNASeqClassifier, DNADataset, train_model, validate_model
import pickle
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

In [2]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

def setup_data_loaders(dataframe):
    dataset = DNADataset(dataframe)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)
    return train_loader, test_loader

def run_notebook(notebook_path):
    with open(notebook_path) as f:
        nb = nbformat.read(f, as_version=4)
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
    try:
        ep.preprocess(nb, {'metadata': {'path': 'directory_containing_notebook/'}})
        print("Notebook executed successfully.")
    except Exception as e:
        print("Error executing the notebook.")
        print(e)
        raise e

In [3]:
pkl_file = 'encoded_df.pkl'
if not os.path.exists(pkl_file):
    print(f"{pkl_file} not found. Running DataPreparation.ipynb...")
    run_notebook('DataPreparation.ipynb')

print("Loading encoded data from Data Preparation Step...")
encoded_df = load_data(pkl_file)
print("Data loaded successfully.")

print("Setting up data loaders...")
train_loader, test_loader = setup_data_loaders(encoded_df)
print("Data loaders ready.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")

model = DNASeqClassifier(sequence_length=800).to(device)

criterion = torch.nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

Loading encoded data from Data Preparation Step...
Data loaded successfully.
Setting up data loaders...
Data loaders ready.
Using: cuda


In [4]:
# Whether to save the model as a file or not
saveModel = True

print("Starting training...")
train_model(model, train_loader, criterion, optimizer, device, num_epochs=20)
print("Training completed.")

print("Starting validation...")
validate_model(model, test_loader, criterion, device)
print("Validation completed.")

if saveModel:
    print("Saving the model...")
    torch.save(model.state_dict(), 'dna_seq_classifier.pth')
    print("Model saved successfully.")

Starting training...


Epoch 1/20: 100%|██████████| 3116/3116 [01:24<00:00, 36.98it/s, loss=0.0191] 


Epoch 1/20, Loss: 0.0631


Epoch 2/20: 100%|██████████| 3116/3116 [01:22<00:00, 37.79it/s, loss=0.0108]  


Epoch 2/20, Loss: 0.0297


...

Epoch 19/20, Loss: 0.0039


Epoch 20/20: 100%|██████████| 3116/3116 [01:23<00:00, 37.21it/s, loss=0.000664]


Epoch 20/20, Loss: 0.0036
Training completed.
Starting validation...


Validating: 100%|██████████| 779/779 [00:06<00:00, 118.65it/s, accuracy=99.60%, loss=0.00867] 


Validation Loss: 0.0118, Accuracy: 99.60%
Validation completed.
Saving the model...
Model saved successfully.
